# Homework 3
#### Team: Jair Aguirre and William Gottschalk, Friday Section

In [1]:
import pandas as pd
from collections import Counter
from sklearn.preprocessing import Imputer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from scipy.stats import zscore
%matplotlib inline

In [2]:
X = pd.read_table("trainingData.txt", header=None)
Y = pd.read_table("trainingTruth.txt", header=None)
print(X.shape)

truth_counts = Counter(Y[0])
truth_percent = {}

for n, count in truth_counts.items():
    truth_percent[n] = count / sum(truth_counts.values())

print(truth_percent)

(17378, 334)
{1: 0.3261019680055242, 2: 0.2240188744389458, 3: 0.3002071584762343, 4: 0.14967199907929565}


In [31]:
imp = Imputer(missing_values='NaN', strategy='median', axis=0)
filled_x = pd.DataFrame(imp.fit_transform(X))
std_norm_x = pd.DataFrame(zscore(filled_x))


X_train, X_test, y_train, y_test = train_test_split(
    std_norm_x.values, Y.values, test_size=0.5, random_state=0
)
r, c = y_train.shape
y_train = y_train.reshape(r,)

r1, c1 = y_test.shape
y_test = y_test.reshape(r1)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(8689, 334) (8689, 334) (8689,) (8689,)


In [27]:
import time
start = time.time()

tuned_parameters = [
    {
        'estimator__kernel': ['rbf'],
        'estimator__gamma': [8e-5, 1e-4],
        'estimator__C': [14, 15, 16, 17, 18, 19, 20]
    }
]

model_to_set = OneVsRestClassifier(SVC(), n_jobs=-1)
clf = GridSearchCV(model_to_set, tuned_parameters)

clf.fit(X_train, y_train)

end = time.time()
print(end - start)

133.39669704437256


In [28]:
 print("The best parameters are %s with a score of %0.2f"
      % (clf.best_params_, clf.best_score_))
pd.DataFrame(clf.cv_results_).sort_values("rank_test_score", axis=0)

The best parameters are {'estimator__gamma': 0.0001, 'estimator__C': 15, 'estimator__kernel': 'rbf'} with a score of 0.74


,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_estimator__C,param_estimator__gamma,param_estimator__kernel,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
3,0.841599,0.783177,0.740982,0.856681,15,0.0001,rbf,"{'estimator__gamma': 0.0001, 'estimator__C': 1...",1,0.741379,0.859447,0.736479,0.864134,0.745098,0.846463,0.048451,0.007710,0.003529,0.007474
8,0.844222,0.769047,0.740599,0.853611,18,8e-05,rbf,"{'estimator__gamma': 8e-05, 'estimator__C': 18...",2,0.740230,0.856567,0.735328,0.860104,0.746251,0.844163,0.050745,0.019062,0.004465,0.006835
10,0.844505,0.767659,0.740215,0.855914,19,8e-05,rbf,"{'estimator__gamma': 8e-05, 'estimator__C': 19...",3,0.737931,0.858295,0.736479,0.863558,0.746251,0.845888,0.047704,0.001661,0.004303,0.007407
11,0.843200,0.744413,0.739831,0.866658,19,0.0001,rbf,"{'estimator__gamma': 0.0001, 'estimator__C': 1...",4,0.737931,0.868664,0.736479,0.875072,0.745098,0.856239,0.053437,0.008078,0.003766,0.007818
1,0.813324,0.785518,0.739447,0.853611,14,0.0001,rbf,"{'estimator__gamma': 0.0001, 'estimator__C': 1...",5,0.739080,0.856567,0.735328,0.858952,0.743945,0.845313,0.003918,0.006979,0.003526,0.005947
6,0.803820,0.775112,0.739064,0.850541,17,8e-05,rbf,"{'estimator__gamma': 8e-05, 'estimator__C': 17...",6,0.739080,0.852535,0.735328,0.856649,0.742791,0.842438,0.001815,0.009105,0.003046,0.005971
13,0.810360,0.752957,0.739064,0.868193,20,0.0001,rbf,"{'estimator__gamma': 0.0001, 'estimator__C': 2...",6,0.739080,0.870392,0.736479,0.875072,0.741638,0.859114,0.002179,0.005816,0.002105,0.006698
5,0.808464,0.756773,0.738296,0.859367,16,0.0001,rbf,"{'estimator__gamma': 0.0001, 'estimator__C': 1...",8,0.736782,0.861175,0.736479,0.868164,0.741638,0.848764,0.003893,0.013645,0.002363,0.008022
7,0.805452,0.751492,0.738296,0.861861,17,0.0001,rbf,"{'estimator__gamma': 0.0001, 'estimator__C': 1...",8,0.736782,0.862903,0.734177,0.870466,0.743945,0.852214,0.003172,0.007675,0.004128,0.007488
12,0.876162,0.758228,0.738296,0.857833,20,8e-05,rbf,"{'estimator__gamma': 8e-05, 'estimator__C': 20...",8,0.735632,0.860599,0.735328,0.867012,0.743945,0.845888,0.050435,0.015309,0.003990,0.008843


In [37]:
model = OneVsRestClassifier(SVC(gamma=0.0001, C=15.0))
model.fit(X_train, y_train)

print('done fitting...')

y_predict = clf.predict(X_test)

print('done predicting...')
report = classification_report(y_test, y_predict)
print(report)

done fitting...
done predicting...
             precision    recall  f1-score   support

          1       0.80      0.91      0.85      2844
          2       0.63      0.63      0.63      1938
          3       0.75      0.90      0.82      2601
          4       0.52      0.15      0.23      1306

avg / total       0.70      0.73      0.70      8689



In [20]:
# original report of running a basic SVM with no param tuning
print(report)

             precision    recall  f1-score   support

          1       0.83      0.92      0.87      3425
          2       0.66      0.69      0.67      2315
          3       0.78      0.93      0.84      3130
          4       0.57      0.16      0.25      1557

avg / total       0.74      0.76      0.73     10427

